In [6]:
import pandas as pd

# Load datasets
ap_top_25 = pd.read_csv("data/REF _ Top 12 in AP Top 25 During Week 6.csv")
conference_mapping = pd.read_csv("data/REF _ NCAAM Conference and ESPN Team Name Mapping.csv")
coaches = pd.read_csv("data/REF _ Current NCAAM Coaches (2025).csv")
postseason_teams = pd.read_csv("data/REF _ Post-Season Tournament Teams.csv")
kenpom_offense = pd.read_csv("data/INT _ KenPom _ Offense.csv")
kenpom_defense = pd.read_csv("data/INT _ KenPom _ Defense.csv")
kenpom_efficiency = pd.read_csv("data/INT _ KenPom _ Efficiency.csv")
kenpom_summary = pd.read_csv("data/INT _ KenPom _ Summary.csv")
kenpom_height = pd.read_csv("data/INT _ KenPom _ Height.csv")
kenpom_misc = pd.read_csv("data/INT _ KenPom _ Miscellaneous Team Stats.csv")
march_madness = pd.read_csv("data/DEV _ March Madness.csv")

# Standardizing column names across all datasets
def standardize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("?", "")
    return df

datasets = {
    "ap_top_25": ap_top_25,
    "conference_mapping": conference_mapping,
    "coaches": coaches,
    "postseason_teams": postseason_teams,
    "kenpom_offense": kenpom_offense,
    "kenpom_defense": kenpom_defense,
    "kenpom_efficiency": kenpom_efficiency,
    "kenpom_summary": kenpom_summary,
    "kenpom_height": kenpom_height,
    "kenpom_misc": kenpom_misc,
    "march_madness": march_madness,
}

# Apply standardization to all datasets
datasets = {name: standardize_columns(df) for name, df in datasets.items()}

# Rename team name columns to be consistent
conference_mapping = conference_mapping.rename(columns={"teamname": "team_name"})
kenpom_offense = kenpom_offense.rename(columns={"teamname": "team_name"})
kenpom_defense = kenpom_defense.rename(columns={"teamname": "team_name"})
kenpom_summary = kenpom_summary.rename(columns={"teamname": "team_name"})
kenpom_height = kenpom_height.rename(columns={"teamname": "team_name"})
kenpom_misc = kenpom_misc.rename(columns={"teamname": "team_name"})
kenpom_efficiency = kenpom_efficiency.rename(columns={"team": "team_name"})
ap_top_25 = ap_top_25.rename(columns={"mapped_espn_team_name": "team_name"})
march_madness = march_madness.rename(columns={"mapped_espn_team_name": "team_name"})
postseason_teams = postseason_teams.rename(columns={"team_name": "team_name"})

# Creating a mapping dictionary to standardize team names
team_name_mapping = {
    "Alabama St.": "Alabama State",
    "Alcorn St.": "Alcorn State",
    "Arizona St.": "Arizona State",
    "Arkansas St.": "Arkansas State",
    "Ball St.": "Ball State",
    "Boise St.": "Boise State",
    "Cal St. Bakersfield": "Cal State Bakersfield",
    "Cal St. Fullerton": "Cal State Fullerton",
    "Chicago St.": "Chicago State",
    "Cleveland St.": "Cleveland State",
    "Colorado St.": "Colorado State",
    "Florida St.": "Florida State",
    "Fresno St.": "Fresno State",
    "Georgia St.": "Georgia State",
    "Idaho St.": "Idaho State",
    "Illinois St.": "Illinois State",
    "Indiana St.": "Indiana State",
    "Iowa St.": "Iowa State",
    "Jackson St.": "Jackson State",
    "Kansas St.": "Kansas State",
    "Kent St.": "Kent State",
    "Long Beach St.": "Long Beach State",
    "Louisiana Monroe": "UL Monroe",
    "McNeese St.": "McNeese State",
    "Miami OH": "Miami (OH)",
    "Michigan St.": "Michigan State",
    "Mississippi St.": "Mississippi State",
    "Missouri St.": "Missouri State",
    "Montana St.": "Montana State",
    "New Mexico St.": "New Mexico State",
    "North Dakota St.": "North Dakota State",
    "Ohio St.": "Ohio State",
    "Oklahoma St.": "Oklahoma State",
    "Oregon St.": "Oregon State",
    "Penn": "Pennsylvania",
    "Penn St.": "Penn State",
    "Portland St.": "Portland State",
    "San Diego St.": "San Diego State",
    "San Jose St.": "San Jose State",
    "South Carolina St.": "South Carolina State",
    "South Dakota St.": "South Dakota State",
    "Texas A&M Corpus Chris": "Texas A&M-Corpus Christi",
    "Texas St.": "Texas State",
    "Utah St.": "Utah State",
    "Washington St.": "Washington State",
    "Weber St.": "Weber State",
    "Wichita St.": "Wichita State",
    "Wright St.": "Wright State",
    "Youngstown St.": "Youngstown State",
}

# Apply team name standardization
datasets_to_standardize = [
    conference_mapping, kenpom_offense, kenpom_defense, kenpom_summary,
    kenpom_height, kenpom_misc, kenpom_efficiency, ap_top_25,
    march_madness, postseason_teams
]

for dataset in datasets_to_standardize:
    dataset["team_name"] = dataset["team_name"].replace(team_name_mapping)

# Rename 'team' to 'team_name' in coaches dataset to match other datasets
coaches = coaches.rename(columns={"team": "team_name"})

# Reattempt merging with corrected column names
merged_data = (
    march_madness
    .merge(conference_mapping, on="team_name", how="left")
    .merge(coaches, on="team_name", how="left")  # Now correctly named
    .merge(postseason_teams, on=["season", "team_name"], how="left")
    .merge(kenpom_offense, on=["season", "team_name"], how="left")
    .merge(kenpom_defense, on=["season", "team_name"], how="left")
    .merge(kenpom_summary, on=["season", "team_name"], how="left")
    .merge(kenpom_efficiency, on=["season", "team_name"], how="left")
    .merge(kenpom_height, on=["season", "team_name"], how="left")
    .merge(kenpom_misc, on=["season", "team_name"], how="left")
)

# Handling missing values
for col in merged_data.columns:
    if merged_data[col].dtype == "object":
        merged_data[col].fillna("Unknown", inplace=True)
    else:
        merged_data[col].fillna(merged_data[col].mean(), inplace=True)

# Save the cleaned dataset
merged_data.to_csv("data/cleaned_march_madness.csv", index=False)

# Print preview of cleaned data
print(merged_data.head())


C:\Users\coope\AppData\Local\Temp\ipykernel_3388\2985911855.py:135: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data[col].fillna(merged_data[col].mean(), inplace=True)
C:\Users\coope\AppData\Local\Temp\ipykernel_3388\2985911855.py:133: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

   season short_conference_name  adjusted_temo_x  adjusted_tempo_rank_x  \
0    2025                   ACC             65.7                    266   
1    2015                   SEC             62.4                    274   
2    2021                   WCC             73.8                      7   
3    2024                    BE             64.6                    330   
4    2025                   SEC             69.6                     59   

   raw_tempo_x  raw_tempo_rank_x  adjusted_offensive_efficiency_x  \
0         66.6               256                            128.0   
1         63.8               242                            121.3   
2         74.3                14                            126.4   
3         66.0               305                            127.5   
4         70.8                44                            128.6   

   adjusted_offensive_efficiency_rank_x  raw_offensive_efficiency_x  \
0                                     3                       1